In [8]:
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']

class Decompressor:
    def __init__(self):
        if not os.path.exists(DECOMPRESS_PATH):
            os.mkdir(DECOMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
    
    def total_decompress(self, compressed_data: str, q_step: np.float64, min_val: np.float64) -> np.ndarray:
        z_decoded = self.entropy_compressor.decompress(bytes.fromhex(compressed_data))
        print('huffman decoded')
        dequantized = self.quantizator.dequantize(z_decoded, q_step, min_val)
        print('dequantized')
        decompressed = self.wavelet_compressor.inverse_wavelet_transform(dequantized)

        return decompressed

    def load_to_file(self, data: List[np.ndarray | np.float64], file_path: str, header_data: dict[str]) -> None:
        z_compressed, q_step, min_val = data
        decompressed = self.total_decompress(z_compressed, q_step, min_val)
        
    def read_mesh_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        for key in MESH_KEYS:
            data[key] = None
        i = 0
        with open(file_path, mode='r') as f:
            while True:
                # Read the first line
                quantilization_keys = f.readline().strip()
                # Read the second line
                huffman_compressed = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys or not huffman_compressed:
                    break
                
                q_step, min_val = map(float, quantilization_keys.split())
                data[MESH_KEYS[i]] = [huffman_compressed, q_step, min_val]
                i += 1
        return data
    
    def read_pturho_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        i = 0
        with open(file_path, mode='r') as f:
            times = f.readline().strip().split(';')
            for key in times:
                data[key] = None
            while True:
                quantilization_keys_p = f.readline().strip()
                huffman_compressed_p = f.readline().strip()

                quantilization_keys_t = f.readline().strip()
                huffman_compressed_t = f.readline().strip()

                quantilization_keys_u = f.readline().strip()
                huffman_compressed_u = f.readline().strip()

                quantilization_keys_rho = f.readline().strip()
                huffman_compressed_rho = f.readline().strip()

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys_p or not huffman_compressed_p:
                    break
                
                q_step_p, min_val_p = map(float, quantilization_keys_p.split())
                q_step_t, min_val_t = map(float, quantilization_keys_t.split())
                q_step_u, min_val_u = map(float, quantilization_keys_u.split())
                q_step_rho, min_val_rho = map(float, quantilization_keys_rho.split())
                data[times[i]] = [
                    [huffman_compressed_p, q_step_p, min_val_p],
                    [huffman_compressed_t, q_step_t, min_val_t],
                    [huffman_compressed_u, q_step_u, min_val_u],
                    [huffman_compressed_rho, q_step_rho, min_val_rho],
                ]
                i += 1
        return data

    def copy_back_not_compressed_piece(self, file_path: str, destination_path: str) -> None:
        # shutil
        pass

In [11]:
decompressor = Decompressor()
mesh_data = decompressor.read_mesh_file('compressed/mesh')
mesh_data

{'faces': ['789c55956d4c975518c64f51ac09abb6e483cee5d3d0dadf39576abc388b636b349b50319dccc21e454b6799abe8c5463b8d19965f580e97d8e4c1b1fd713a872f4981e423e00b22a1283951f291141414753275b6d90bd7af0ffdbf5cbbafebbeef73cefd9c73fd8df9e7975167ff05b366bfd01e15aefe5538d4289c7068045d67abe22f0f0b1ffb59fcb8038aab54ef06db15173409bd36f12f809f1f14ff76285cabd885d4b57508b7340b3be85bac7db9d83161a674b7f88830e7f8088679271497d17703eb0e49370fa137ea3cd62a36afc177abbf09754e7799f5ca3ad5bffca4e2d7e95ba8bab01ffd7dfacc664eb3986bbbeaccf4d3c272d59bf13ab73d7e46f1d448f167e74730dad52b7ec525f54d063f95ee2f951ed65f54de17423f537dcc007a8aea6c23790b2f8ca0b79678b9f4e8e61fca8f9fa58e3e7794e7bdd827be4f7cf4bdf2dd5dd5fbbbe853aafe2e437576097549c2a05f7a1097ee6ac5bb34a16d515fbb1d7e65bfe2998a4da2e61596f448afd4baa659797e11eb162876e32f8bbf46fd44f50fbad8f703d2a31ae57bb3d09ba57bf9d2bd25c2f01de6fc97f2cd9be26daafafb2b351f57d82b7e0afacb03ea13a93ed8cc7edfbba2f5bb59a7497af8613f79d24d36795b39df7dd6fb137e85fabb22ce398e78ccc0ffcee76e6b5f2ec6f7be29dd